In [2]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('data/BTC-USDT.csv')

In [ ]:
df

,date,open,high,low,close,volume
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...,...
51404,1688364000000,30673.61,30678.41,30606.00,30651.25,1036.522350
51405,1688367600000,30651.25,30759.80,30636.65,30667.19,1758.796390
51406,1688371200000,30667.19,30700.92,30599.07,30640.12,1390.909010
51407,1688374800000,30640.11,30670.00,30602.29,30610.61,833.620010


In [19]:
df['date'] = pd.to_datetime(df['date'], unit='ms')
df = df.set_index(df['date'])
del df['date']

In [6]:
df

,open,high,low,close,volume
date,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2023-07-03 06:00:00,30673.61,30678.41,30606.00,30651.25,1036.522350
2023-07-03 07:00:00,30651.25,30759.80,30636.65,30667.19,1758.796390
2023-07-03 08:00:00,30667.19,30700.92,30599.07,30640.12,1390.909010


In [20]:
df['ma50'] = df['close'].rolling(50).mean()
df['ma200'] = df['close'].rolling(200).mean()

In [33]:
df['buy_signal'] = False
df['sell_signal'] = False

df.loc[(df['ma50'] > df['ma200']), 'buy_signal'] = True
df.loc[(df['ma50'] < df['ma200']), 'sell_signal'] = True


In [34]:
balance = 1000
position = None
asset = 'BTC'


In [35]:
for index, row in df.iterrows():

    if position is None and row['buy_signal']:
        open_price = row['close']
        open_usd_size = balance

        position = {
            'open_price': open_price,
            'usd_size': open_usd_size
        }

        print(f'{index}  Buy for {open_usd_size} of {asset} at {open_price}' )

    elif position and row['sell_signal']:
        close_prise = row['close']
        trade_result = (close_prise - position['open_price']) / position['open_price']
        balance = balance + trade_result * position['usd_size']
        position = None
        print(f'{index}  Sell for {open_usd_size} of {asset} at {open_price}' )


print(f'Final balance : {balance}')

2017-08-25 11:00:00  Buy for 1000 of BTC at 4361.27
2017-09-04 06:00:00  Sell for 1000 of BTC at 4361.27
2017-09-07 21:00:00  Buy for 1022.5851644131182 of BTC at 4641.13
2017-09-09 13:00:00  Sell for 1022.5851644131182 of BTC at 4641.13
2017-09-19 09:00:00  Buy for 947.4588776015138 of BTC at 3940.2
2017-09-22 21:00:00  Sell for 947.4588776015138 of BTC at 3940.2
2017-09-26 13:00:00  Buy for 871.4760516705198 of BTC at 3928.33
2017-10-05 07:00:00  Sell for 871.4760516705198 of BTC at 3928.33
2017-10-06 17:00:00  Buy for 921.5617492024618 of BTC at 4336.16
2017-10-19 19:00:00  Sell for 921.5617492024618 of BTC at 4336.16
2017-10-20 15:00:00  Buy for 1192.0777488092249 of BTC at 5889.99
2017-10-25 02:00:00  Sell for 1192.0777488092249 of BTC at 5889.99
2017-10-27 21:00:00  Buy for 1072.4670145348434 of BTC at 5769.6
2017-10-28 17:00:00  Sell for 1072.4670145348434 of BTC at 5769.6
2017-10-29 21:00:00  Buy for 1059.5983769488328 of BTC at 6132.59
2017-11-10 12:00:00  Sell for 1059.598376

In [56]:
class BackTestCrossMA:

    def __init__(self):
        self.df = pd.DataFrame()

    def __transform_data(self):
        
        self.df['date'] = pd.to_datetime(self.df['date'], unit='ms')
        self.df = self.df.set_index(self.df['date'])
        # del self.df['date']


    def load_data(self, path):
        self.df = pd.read_csv(path)
        self.__transform_data()
    
    def populate_indicators(self):
        self.df['ma50'] = self.df['close'].rolling(50).mean()
        self.df['ma200'] = self.df['close'].rolling(200).mean()
        
    def populate_signals(self):
        self.df['buy_signal'] = False
        self.df['sell_signal'] = False

        self.df.loc[(df['ma50'] > self.df['ma200']), 'buy_signal'] = True
        self.df.loc[(df['ma50'] < self.df['ma200']), 'sell_signal'] = True

    def run_backtest(self):

        balance = 1000
        position = None
        asset = 'BTC'
        
        for index, row in df.iterrows():

            if position is None and row['buy_signal']:
                open_price = row['close']
                open_usd_size = balance

                position = {
                    'open_price': open_price,
                    'usd_size': open_usd_size
                }

                print(f'{index}  Buy for {open_usd_size} of {asset} at {open_price}' )

            elif position and row['sell_signal']:
                close_prise = row['close']
                trade_result = (close_prise - position['open_price']) / position['open_price']
                balance = balance + trade_result * position['usd_size']
                position = None
                print(f'{index}  Sell for {open_usd_size} of {asset} at {open_price}' )


        print(f'Final balance : {balance}')


bt = BackTestCrossMA()
bt.load_data('data/BTC-USDT.csv')
bt.populate_indicators()
bt.populate_signals()
bt.run_backtest()


2017-08-25 11:00:00  Buy for 1000 of BTC at 4361.27
2017-09-04 06:00:00  Sell for 1000 of BTC at 4361.27
2017-09-07 21:00:00  Buy for 1022.5851644131182 of BTC at 4641.13
2017-09-09 13:00:00  Sell for 1022.5851644131182 of BTC at 4641.13
2017-09-19 09:00:00  Buy for 947.4588776015138 of BTC at 3940.2
2017-09-22 21:00:00  Sell for 947.4588776015138 of BTC at 3940.2
2017-09-26 13:00:00  Buy for 871.4760516705198 of BTC at 3928.33
2017-10-05 07:00:00  Sell for 871.4760516705198 of BTC at 3928.33
2017-10-06 17:00:00  Buy for 921.5617492024618 of BTC at 4336.16
2017-10-19 19:00:00  Sell for 921.5617492024618 of BTC at 4336.16
2017-10-20 15:00:00  Buy for 1192.0777488092249 of BTC at 5889.99
2017-10-25 02:00:00  Sell for 1192.0777488092249 of BTC at 5889.99
2017-10-27 21:00:00  Buy for 1072.4670145348434 of BTC at 5769.6
2017-10-28 17:00:00  Sell for 1072.4670145348434 of BTC at 5769.6
2017-10-29 21:00:00  Buy for 1059.5983769488328 of BTC at 6132.59
2017-11-10 12:00:00  Sell for 1059.598376

In [69]:
class CrossMAStrategy:

    def __init__(self, df: pd.DataFrame):
        self.df = df

    def __populate_indicators(self):

        self.df['ma50'] = self.df['close'].rolling(50).mean()
        self.df['ma200'] = self.df['close'].rolling(200).mean()
        
    def __populate_signals(self):
        self.df['buy_signal'] = False
        self.df['sell_signal'] = False

        self.df.loc[(self.df['ma50'] > self.df['ma200']), 'buy_signal'] = True
        self.df.loc[(self.df['ma50'] < self.df['ma200']), 'sell_signal'] = True

    def isSell(self, index):
        return   bool(self.df.loc[index]['sell_signal'])
    
    def isBuy(self, index):
        return   bool(self.df.loc[index]['buy_signal'])
    
    def init(self):
        self.__populate_indicators()
        self.__populate_signals()
        

class BackTestCrossMA:

    def __init__(self):
        self.df = pd.DataFrame()

    def __transform_data(self):
        
        self.df  = self.df.drop_duplicates(subset=['date'], keep='first')

        self.df['date'] = pd.to_datetime(self.df['date'], unit='ms')
        self.df = self.df.set_index(self.df['date'])
        del self.df['date']

        self.strategy = CrossMAStrategy(self.df)
        self.strategy.init()
        

    def load_data(self, path):
        self.df = pd.read_csv(path)
        self.__transform_data()
    
   
    def run_backtest(self):

        balance = 1000
        position = None
        asset = 'BTC'
        
        for index, row in df.iterrows():

            if not position and self.strategy.isBuy(index):
                open_price = row['close']
                open_usd_size = balance

                position = {
                    'open_price': open_price,
                    'usd_size': open_usd_size
                }

                print(f'{index}  Buy for {open_usd_size} of {asset} at {open_price}' )

            elif position and self.strategy.isSell(index):
                close_prise = row['close']
                trade_result = (close_prise - position['open_price']) / position['open_price']
                balance = balance + trade_result * position['usd_size']
                position = None
                print(f'{index}  Sell for {open_usd_size} of {asset} at {open_price}' )


        print(f'Final balance : {balance}')


bt = BackTestCrossMA()
bt.load_data('data/BTC-USDT.csv')
bt.run_backtest()


2017-08-25 11:00:00  Buy for 1000 of BTC at 4361.27
2017-09-04 06:00:00  Sell for 1000 of BTC at 4361.27
2017-09-07 21:00:00  Buy for 1022.5851644131182 of BTC at 4641.13
2017-09-09 13:00:00  Sell for 1022.5851644131182 of BTC at 4641.13
2017-09-19 09:00:00  Buy for 947.4588776015138 of BTC at 3940.2
2017-09-22 21:00:00  Sell for 947.4588776015138 of BTC at 3940.2
2017-09-26 13:00:00  Buy for 871.4760516705198 of BTC at 3928.33
2017-10-05 07:00:00  Sell for 871.4760516705198 of BTC at 3928.33
2017-10-06 17:00:00  Buy for 921.5617492024618 of BTC at 4336.16
2017-10-19 19:00:00  Sell for 921.5617492024618 of BTC at 4336.16
2017-10-20 15:00:00  Buy for 1192.0777488092249 of BTC at 5889.99
2017-10-25 02:00:00  Sell for 1192.0777488092249 of BTC at 5889.99
2017-10-27 21:00:00  Buy for 1072.4670145348434 of BTC at 5769.6
2017-10-28 17:00:00  Sell for 1072.4670145348434 of BTC at 5769.6
2017-10-29 21:00:00  Buy for 1059.5983769488328 of BTC at 6132.59
2017-11-10 12:00:00  Sell for 1059.598376